---
# 3.2 Bisection method
---

Suppose $f \in C[a,b]$ and that $f(a)$ and $f(b)$ have opposite signs; i.e.,

$$
f(a) \cdot f(b) < 0.
$$

Recall the following important theorem from calculus.

> ### Intermediate Value Theorem
>
> If $f \in C[a,b]$ and $f(a) \leq s \leq f(b)$, then there exists a real number $c \in [a,b]$ such that $f(c) = s$.

Since $f$ changes sign over $[a,b]$, the Intermediate Value Theorem implies that there is some $x^* \in [a,b]$ such that $f(x^*) = 0$.

The **bisection method** searches for a root of $f$ in $[a,b]$ as follows.

1. Let $p = (a+b)/2$ be the **midpoint** of $[a,b]$.
2. If $f(a) \cdot f(p) < 0$, then there is a root in $[a,p]$.
3. If $f(a) \cdot f(p) = 0$, then $p$ is a root.
4. If $f(a) \cdot f(p) > 0$, then there is a root in $[p,b]$.

Each time we apply the above, we get a subinterval that contains a root that is **half the size** of the interval $[a,b]$.

In [ ]:
# Code the bisection method

function bisect(f, a, b; ftol=eps(), tol=eps(), maxiter=2000, verbose=false)
    
    a >= b && error("Need to have a < b.")
    
    fa, fb = f(a), f(b)
    
    if abs(fa) <= ftol
        @info("abs(f(a)) <= ftol")
        return a
    elseif abs(fb) <= ftol
        @info("abs(f(b)) <= ftol")
        return b
    end
    
    if signbit(fa) == signbit(fb)
        error("The signs of f(a) and f(b) must be different.")
    end

    k = 0
    p = 0.0
    while true
        k += 1
                
        p = 0.5a + 0.5b
        fp = f(p)
        
        # Check stopping conditions
        if (0.5*(b - a) <= tol*(1.0 + abs(p))
                || abs(fp) <= ftol
                || nextfloat(a) == b)
            @info("Success.")
            break
        elseif k == maxiter
            @warn("Failed. Increase maxiter.")
            break
        end
        
        if signbit(fa) != signbit(fp)  # Root is in [a, p]
            b = p
        else                           # Root is in [p, b]
            a = p
        end
    end
    
    @show a, p, b, k
    
    return p
end

In [ ]:
f(x) = x
a, b = -2.0, 1.0

p = bisect(f, a, b, ftol=0.0, tol=0.0)
p, f(p)

In [ ]:
f(x) = sin(x)
a, b = -1.0, 2.0

p = bisect(f, a, b, ftol=0.0, tol=0.0)
p, f(p)

In [ ]:
f(x) = sin(x)
a, b = 3.0, 4.0

p = bisect(f, a, b, ftol=0.0, tol=0.0)
p, f(p)

In [ ]:
bitstring(p)

In [ ]:
bitstring(float(π))

In [ ]:
# ChatGPT: "write a function in Julia that uses the bisection method to find a root of f(x) in the interval [a,b]"

function bisection(f, a, b, tol)
    if f(a) * f(b) > 0
        error("The function must have opposite signs at the interval endpoints.")
    end

    while (b - a) / 2 > tol
        c = (a + b) / 2
        if f(c) == 0
            return c
        elseif f(c) * f(a) < 0
            b = c
        else
            a = c
        end
    end

    return (a + b) / 2
end

In [ ]:
f(x) = 2cosh(x/4) - x
a, b = 8.0, 10.0

p = bisection(f, a, b, 1e-15)

p, f(p)

---

### Example

We will now use the above `bisect` function to find the root of

$$
f(x) = 2 \cosh(x/4) - x
$$

that lies in the interval $[5,10]$.

In [ ]:
f(x) = 2cosh(x/4) - x
a, b = 5.0, 10.0

p = bisect(f, a, b, tol=0.0)
p, f(p)

---

### Example

Let's try to find the value of $y_0$ such that the solution $y(t)$ of the ordinary differential equation 

$$
\frac{dy}{dt} = t - 5 + \frac{y^2}{100}
$$

with initial value $y(0) = y_0$ satisfies $y(10) = 20$.

In [ ]:
using OrdinaryDiffEq  # Package for solving ordinary differential equations
using Plots, LaTeXStrings

In [ ]:
f(y, p, t) = t - 5 + y^2/100

function foo(y0)
    tspan = (0.0, 10.0)
    prob = ODEProblem(f, y0, tspan)
    sol = solve(prob, Tsit5())
end

sol = foo(y0)

In [ ]:
plot(foo(0.0), label=:none)
plot!(foo(5.0), label=:none)
plot!(foo(10.0), label=:none)
plot!(foo(15.0), label=:none)
ylims!(-10, 50)

In [ ]:
function bar(y0)
    sol = foo(y0)
    return sol(10.0) - 20.0
end

In [ ]:
bar(10.0)

In [ ]:
bar(15.0)

In [ ]:
y0sol = bisect(bar, 10.0, 15.0)

In [ ]:
plot(foo(y0sol), label=L"y(t)")
ylims!(0, 30)

In [ ]:
sol = foo(y0sol)

In [ ]:
nextfloat(20.0)

In [ ]:
bitstring(20.000000000000004)

In [ ]:
bitstring(20.000000000000005)

---

## Analyzing the bisection method

Initially, we know a root $x^*$ is somewhere in the interval $[a,b]$. If we let $x_k$ be the midpoint of the $k$th subinterval, then

$$\left|x^* - x_0\right| \leq \frac{b-a}{2}.$$

In the next iteration, 

$$\left|x^* - x_1\right| \leq \frac{b-a}{4},$$

and in the following iteration,

$$\left|x^* - x_2\right| \leq \frac{b-a}{8},$$

and so on, each time reducing our error bound by a factor of $2$.

In general,

$$\left|x^* - x_k\right| \leq \frac{b-a}{2} \cdot 2^{-k}, 
\qquad \text{for $k = 0,1,2,\ldots$}.$$

Suppose we want to compute $x_k$ such that 

$$\left|x^* - x_k\right| \leq \mathtt{atol}.$$

Then we just need to find the smallest positive integer $k$ such that

$$\frac{b-a}{2} \cdot 2^{-k} \leq \mathtt{atol}.$$

That is,

$$\frac{b-a}{2\mathtt{atol}} \leq 2^k,$$

which gives us

$$\log_2\left(\frac{b-a}{2\mathtt{atol}}\right) \leq k,$$

so we just need the first integer $k$ that is larger than $\log_2\left(\frac{b-a}{2\mathtt{atol}}\right)$. Therefore, 

$$k = \left\lceil \log_2\left(\frac{b-a}{2\mathtt{atol}}\right) \right\rceil.$$

In [ ]:
log2(Inf)

In [ ]:
atol = eps()
a, b = 8.0, 11.0
k = ceil(log2((b-a)/2atol))

In [ ]:
log2((b-a)/2atol)

In [ ]:
ceil(1.5)

In [ ]:
ceil(-1.5)

In [ ]:
ceil(10.0)

---

## Pros and cons of the bisection method

Pros:

1. **Simple:** The bisection method only requires function values, is easy to understand and implement, and it is easy to analyze.

2. **Robust:** The bisection method is guaranteed to work, provided that $f$ is continuous and changes sign on the interval $[a,b]$.

Cons:

1. **Slow to converge:** The bisection method often requires many function evaluations.

2. **Does not generalize:** The bisection method only applies to solving equations involving one variable; it does not generalize to solving equations involving multiple variables.



---